In [1]:
import spacy
import pickle
import neuralcoref.train.document as dc

In [2]:
with open('pickleObjects/corefSpansObject', 'rb') as f:
    corefSpans = pickle.load(f)

In [3]:
numOfMinus = 0
for i in corefSpans:
    for j in i:
        for k in j:
            if k == -1 and len(i) < 3:
                numOfMinus += 1
                break

In [4]:
sweCorp = ''

with open('../Data/Datasets/Europarl/Documents/ep-09-03-11-018/text/ep-09-03-11-018-sv.txt') as sweDoc:
    for line in sweDoc:
        sweCorp = sweCorp + line



In [5]:
nlp = spacy.load('../Models/SwedishModel')
doc = nlp(sweCorp)

In [6]:
numOfSingCoref = 0
corefSpansSwe = []
numOfCorefs = 0
for clusterSub in corefSpans:
    corefClusterSwe = []
    for ment in clusterSub:
        corefSwe = []
        startOfLine = 0
        endOfLine = 0
        lineText = ''
        mentionText = ''
        if ment[0] != -1:
            startOfLine = ment[0]
            endOfLine = ment[-1]
            while doc[startOfLine].text != '\n' and startOfLine != 0:
                startOfLine -= 1
            startOfLine += 1
            while doc[endOfLine].text != '\n':
                endOfLine += 1

            for tok in range(startOfLine, endOfLine):
                lineText = lineText + doc[tok].text + ' '
            lineText.rstrip()
        else:
            continue
            
        #mentionSpanDoc = nlp(mentionText)
        lineDoc = nlp(lineText)

        #mentionsInSpan = dc.extract_mentions_spans(mentionSpanDoc, [])
        mentionsInLine = dc.extract_mentions_spans(lineDoc, [])

        mentionsInSpan = []

        for span in mentionsInLine:
            if (span.start + startOfLine) >= ment[0] and (span.end + startOfLine) <= ment[-1]:
                mentionsInSpan.append(span)

        if len(mentionsInSpan) == 0:
            continue

        invalidCoref = False

        """for span in mentionsInLine:
            if (span.end + startOfLine) > mention[1] > (span.start + startOfLine) > mention[0] or (span.start + startOfLine) < mention[0] < (span.end + startOfLine) < mention[1]:
                invalidCoref = True"""

        maxLength = 0
        maxSpan = 0
        for subSpan in mentionsInSpan:
            if subSpan.end - subSpan.start > maxLength:
                maxLength = subSpan.end - subSpan.start
                maxSpan = subSpan

        """for span in mentionsInSpan:
            if span.start < maxSpan.start or span.end > maxSpan.end:
                invalidCoref = True"""

        if invalidCoref:
            continue
        elif not invalidCoref:
            corefSwe.append(maxSpan.start + startOfLine)
            corefSwe.append(maxSpan.end + startOfLine)
            corefClusterSwe.append(corefSwe)

    if len(corefClusterSwe) > 1:
        res = list(set(tuple(sorted(sub)) for sub in corefClusterSwe))
        corefSpansSwe.append(res)
    else:
        numOfSingCoref += 1

In [7]:
noOfCorefsEng = 0
for sublist in corefSpans:
    for span in sublist:
        if span[0] != -1:
            noOfCorefsEng += 1

In [8]:
noOfCorefsSwe = 0
for sublist in corefSpansSwe:
    for span in sublist:
        noOfCorefsSwe += 1

In [9]:
with open('pickleObjects/corefSpansSweObject', 'wb') as f:
    pickle.dump(corefSpansSwe, f)

